In [ ]:
# Ознайомитись з використанням класу Matcher. Ознайомитись із синтаксичними залежностями та їх застосуванням для виявлення намірів.
import json
import spacy
from spacy.matcher import Matcher, PhraseMatcher
import re

# Завантаження моделі spaCy
nlp = spacy.load("en_core_web_sm") # може розбивати текст на токени, визначати частини мови, іменовані сутності (ім’я, дата, місце і т.д.)

# Функція для витягнення імен лікарів
def extract_doctor_names(data):
    matcher = Matcher(nlp.vocab)
    phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER") #attr="LOWER" — означає, що порівняння буде незалежне від регістру (тобто "Dr", "dr", "DR" — однаково).

    doctor_names = set()
    known_doctors = set()

    # Збір імен стоматологів з service_results
    for turn in data.get("turns", []):
        if turn.get("speaker") == "SYSTEM":
            for frame in turn.get("frames", []):
                for result in frame.get("service_results", []):
                    if "dentist_name" in result and result["dentist_name"].strip():
                        name = result["dentist_name"].strip()
                        known_doctors.add(name)

    doctor_names.update(known_doctors)

    if known_doctors:
        dentist_patterns = [nlp(name) for name in known_doctors]
        phrase_matcher.add("KNOWN_DENTIST", dentist_patterns)

    dr_title_pattern = [
        {"LOWER": {"IN": ["dr", "dr.", "doctor"]}},
        {"TEXT": ".", "OP": "?"}, # Опціонально (?) допускається наявність крапки Dr. Smith
        {"IS_ALPHA": True, "IS_TITLE": True, "OP": "+"}, # Alpha - лише з літер, Title - з великох букви, Опціонально (+) - 1 або більше слів (Імена з Прізвищем або без)
        {"IS_ALPHA": True, "LENGTH": 1, "OP": "?"}, {"IS_PUNCT": True, "OP": "?"}, # Опціонально (?) допускаємо наявність одного алфавітного символу (LENGTH: 1), наприклад, ініціал: "J." з розділовим знаком (PUNKT)
        {"IS_ALPHA": True, "IS_TITLE": True, "OP": "+"} # Один або більше слів з літер, які починаються з великої літери.
    ]
    matcher.add("DR_TITLE", [dr_title_pattern])

    for turn in data.get("turns", []):
        utterance = turn.get("utterance", "")
        doc = nlp(utterance)

        pattern_matches = matcher(doc)
        for match_id, start, end in pattern_matches:
            span = doc[start:end]
            if is_valid_doctor_name(span.text):
                doctor_names.add(span.text)

        phrase_matches = phrase_matcher(doc)
        for match_id, start, end in phrase_matches:
            span = doc[start:end]
            doctor_names.add(span.text)

    return doctor_names

# Перевірка валідності імені лікаря
def is_valid_doctor_name(text):
    if len(text) < 5:
        return False
    parts = text.split()

    if parts[0].lower() in ["dr", "dr.", "doctor"]:
        if len(parts) < 2:
            return False
        if not all(part[0].isupper() for part in parts[1:] if part and part[0].isalpha()):
            return False
    else:
        if not all(part[0].isupper() for part in parts if part and part[0].isalpha()):
            return False

    common_false_positives = [
        "okay so", "yes i", "sure i", "well i", "great i", "right i",
        "hello", "thank you", "see you", "help you", "tell you", "okay i",
        "okay let", "okay thank", "okay the", "no i", "hi there", "hi i",
        "second street", "first street", "main street", "oak street", "pine street",
        "city center", "medical center", "dental center", "health center",
        "good morning", "good afternoon", "good evening",
        "sun", "mon", "tue", "wed", "thu", "fri", "sat"
    ]
    if any(fp in text.lower() for fp in common_false_positives):
        return False
    if any(c in text for c in ",;:?!()-"):
        return False
    if any(c.isdigit() for c in text):
        return False
    if " " not in text:
        return False
    return True

# Виявлення підтверджень користувача ----------------------------------------------------------------------------------------------------------------
def find_confirmations(data):
    matcher = Matcher(nlp.vocab)
    confirmation_patterns = [
        [{"LOWER": {"IN": ["yes", "yeah", "yep", "yup"]}}, {"OP": "*"}],
        [{"LOWER": {"IN": ["yes", "yeah", "yep"]}}, {"LOWER": "that"}, {"LOWER": "sounds"}, {"LOWER": "good"}, {"OP": "*"}],
        [{"LOWER": {"IN": ["yes", "yeah", "yep"]}}, {"LOWER": "that"}, {"LOWER": "is"}, {"LOWER": "correct"}, {"OP": "*"}],
        [{"LOWER": "that"}, {"LOWER": "sounds"}, {"LOWER": "good"}, {"OP": "*"}],
        [{"LOWER": "that"}, {"LOWER": "would"}, {"LOWER": "be"}, {"LOWER": {"IN": ["perfect", "great", "excellent"]}}, {"OP": "*"}],
        [{"LOWER": {"IN": ["perfect", "excellent", "great", "wonderful"]}}, {"OP": "*"}],
        [{"LOWER": {"IN": ["correct", "exactly", "absolutely"]}}, {"OP": "*"}],
        [{"LOWER": {"IN": ["right", "ok", "okay", "sure", "fine"]}}, {"OP": "*"}]
        # IN — токен повинен входити в заданий список.
        # OP: "*" — блок опціональний.
    ]
    matcher.add("CONFIRMATION", confirmation_patterns)
    confirmations = []

    for i, turn in enumerate(data.get("turns", [])):
        if turn.get("speaker") == "USER":
            utterance = turn.get("utterance", "")
            doc = nlp(utterance)
            matches = matcher(doc)

            if matches:
                match_span = doc[matches[0][1]:matches[0][2]].text # Індекси токенів збігу

                if len(match_span.split()) > 3:
                    confirmation_keywords = ["yes", "yeah", "correct", "perfect", "right", "okay", "ok", "sure", "absolutely"]
                    for keyword in confirmation_keywords:
                        if keyword in match_span.lower():
                            # ключове слово, яке може бути (опціонально) між двома сусіднімі слови w+ з розідловими знаками s+
                            pattern = re.compile(r'\b(\w+\s+)?' + re.escape(keyword) + r'(\s+\w+)?\b', re.IGNORECASE)
                            match = pattern.search(match_span)
                            if match:
                                match_span = match.group(0) # Повертає весь знайдений фрагмент тексту, який відповідає регулярному виразу.
                                break

                confirmations.append({
                    "turn_index": i,
                    "utterance": utterance,
                    "match": match_span
                })
    return confirmations

# Виявлення намірів користувача через синтаксичний аналіз ----------------------------------------------------------------------------------------------------
def extract_intents_with_deps(data):
    intents = []
    for i, turn in enumerate(data.get("turns", [])):
        if turn.get("speaker") == "USER":
            utterance = turn.get("utterance", "")
            doc = nlp(utterance)

            detected_intent = None
            intent_confidence = 0
            intent_reason = ""

            explicit_intent = None
            for frame in turn.get("frames", []):
                for action in frame.get("actions", []):
                    if action.get("slot") == "intent" and action.get("act") == "INFORM_INTENT":
                        explicit_intent = action.get("canonical_values", [None])[0] # Якщо ключа немає — повертає список із одним елементом None

            for token in doc:
                # Якщо основна форма слова — одне з дієслів, що означають бронювання
                if token.lemma_ in ["book", "make", "schedule", "set", "need", "have"]:
                    # То розглядаються діти токена, тобто граматично залежні слова у реченні від цього дієслова (об’єкта пошуку)
                    for child in token.children:
                        if child.lemma_ in ["appointment", "booking"]: # Якщо є об'єкт типу "appointment" або "booking", то:
                            detected_intent = "BookAppointment"
                            # Встановлюємо намір як "BookAppointment" (опціонально впевненість 0.9 і пояснюємо причину, але краще було б зробити це навченою моделлю).
                            intent_confidence = 0.9 
                            intent_reason = f"Знайдено дієслово '{token.text}' з об'єктом '{child.text}'"
                            break
                elif token.lemma_ in ["find", "look", "search", "need", "want"]:    
                    for child in token.children:
                        # child.dep_ — це тип граматичної залежності, тобто яка роль цього слова в реченні.
                        #   dobj = "I need a doctor" де doctor — це dobj
                        #   pobj = "I am looking for a doctor" де doctor — це pobj (Об'єкт прийменника)
                        #   attr = атрибут, який дає додаткову інформацію. "My doctor is a dentist" де dentist — це attr
                        #   prep = "look for a dentist" де for — це prep
                        # Якщо це слово також є в списку ("dentist", "doctor", "provider" тощо) — ми вважаємо, що користувач шукає лікаря.        
                        if ((child.dep_ in ["dobj", "pobj", "attr"] and child.lemma_ in ["dentist", "doctor", "provider", "specialist", "appointment"]) or
                            (child.dep_ == "prep" and any(grandchild.lemma_ in ["dentist", "doctor", "provider"] for grandchild in child.children))):
                            detected_intent = "FindProvider"
                            intent_confidence = 0.9
                            intent_reason = f"Знайдено пошуковий дієслівний шаблон '{token.text}' з об'єктом '{child.text}'"
                            break
                elif token.lemma_ in ["yes", "yeah", "yep", "correct", "right", "ok", "okay"]: # Якщо є слово-підтвердження
                    #  І це — перше слово у реченні або йде після розділових знаків, тоді:
                    if token.i == 0 or (token.i <= 2 and all(t.is_punct for t in doc[:token.i])):
                        detected_intent = "Confirm"
                        intent_confidence = 0.8
                        intent_reason = f"Початок речення зі словом-підтвердженням '{token.text}'"
                        break
                elif token.lemma_ in ["what", "when", "where", "how", "can", "could", "will", "would", "is", "are"]: # Якщо знайдено запитальне слово або допоміжне дієслово.
                    if token.i == 0 or token.dep_ == "aux" or token.pos_ == "AUX": #  І воно або перше в реченні, або має граматичну роль/частину мови допоміжного
                        detected_intent = "GetInformation"
                        intent_confidence = 0.8
                        intent_reason = f"Знайдено допоміжне/запитальне слово '{token.text}'"
                        break

            if not detected_intent: # Якщо не знайдено жодного наміру раніше
                # Якщо є позитивні прикметники — можемо припустити, що це підтвердження:
                if any(token.lemma_ in ["perfect", "great", "wonderful", "excellent", "good", "fine"] for token in doc):
                    detected_intent = "Confirm"
                    intent_confidence = 0.7
                    intent_reason = "Позитивний прикметник, що ймовірно означає підтвердження"
                # Або бронювання - якщо є розпізнаний час або дата (виявлені spaCy як сутності).
                elif any(token.ent_type_ == "TIME" or token.ent_type_ == "DATE" for token in doc):
                    detected_intent = "BookAppointment"
                    intent_confidence = 0.7
                    intent_reason = "Виявлено часовий вираз — ймовірно бронювання"

            if not detected_intent and explicit_intent:
                detected_intent = explicit_intent
                intent_confidence = 1.0
                intent_reason = "Явно вказаний намір у фреймі"

            if detected_intent:
                intents.append({
                    "turn_index": i,
                    "utterance": utterance,
                    "detected_intent": detected_intent,
                    #"confidence": intent_confidence,
                    # "reason": intent_reason
                })
    return intents

# Основна функція обробки діалогів
def process_dialogue(json_data):
    if isinstance(json_data, str):
        try:
            data = json.loads(json_data)
        except json.JSONDecodeError:
            return {"error": "Невірний JSON"}
    else:
        data = json_data

    if isinstance(data, list) and len(data) > 0:
        results = {
            "doctor_names": set(),
            "confirmations": [],
            "intents": []
        }
        for dialogue in data:
            if "dialogue_id" in dialogue:
                dialogue_results = process_single_dialogue(dialogue)
                results["doctor_names"].update(dialogue_results["doctor_names"])
                results["confirmations"].extend(dialogue_results["confirmations"])
                results["intents"].extend(dialogue_results["intents"])
        results["doctor_names"] = list(results["doctor_names"])
        return results
    elif "dialogue_id" in data or "turns" in data:
        return process_single_dialogue(data)
    else:
        return {"error": "Невідомий формат JSON"}

# Обробка одного діалогу
def process_single_dialogue(dialogue):
    doctor_names = extract_doctor_names(dialogue)
    confirmations = find_confirmations(dialogue)
    intents = extract_intents_with_deps(dialogue)
    return {
        "dialogue_id": dialogue.get("dialogue_id", "unknown"),
        "doctor_names": list(doctor_names),
        "confirmations": confirmations,
        "intents": intents
    }

# Зчитування даних та вивід результатів
with open("services2.json", "r") as f:
    json_data = f.read()
data = json.loads(json_data)
results = process_dialogue(data)

print("Виявлені імена докторів:")
for name in results["doctor_names"]:
    print(f"- {name}")

print("\nПідтвердження від користувача:")
for conf in results["confirmations"]:
    print(f"- Індекс {conf['turn_index']}: '{conf['utterance']}')") # (збіг за: '{conf['match']}'

print("\nЗнайдені наміри:")
for intent in results["intents"]:
    print(f"- Індекс {intent['turn_index']}: '{intent['utterance']}' - Намір: {intent['detected_intent']}") # Причина: {intent['reason']} (Ймовірність: {intent['confidence']:.2f}) 

Виявлені імена докторів:
- Dr. Kwang H. Kim
- Dr. Stephen T. Fan
- Dr. Brian W. Chun
- Chandiok Neena K
- Boyer Dental Arts
- Marenda Ronald E
- Ghina Morad
- Dr. Maria Theresa V. Chua
- Bautista Maximilian
- Dr. Andrei Simel
- A J Dental lab
- Wang Jen-Kuei
- Hart Brad L
- A J Dental Lab
- Centerville Dental Care
- North Bay Smiles
- Dr Pucan Dental
- Gerald e. Dixon Scott E. Dixon
- Donna M Cotner
- Cofield Marianne
- Belle Dental
- Dr Oscar Ventanilla
- Cisneros Rob M
- Steven C Fong
- Edwards John G
- Gilman Dental Group/Valerie Gilman
- D Sign Dental Lab
- Andrei Simel , Family & Cosmetic Dentisry
- Hahn Eun
- Dr. Eugene H. Burton Iii
- Cabanas Carol L
- A-1 Dental Care
- Golden Life Dental Lab
- Hasaimoto Family Dentistry
- Dr. Christopher J. Bennett
- Dr. Keller & Dr. Burk - Antioch Premier Dental
- Petaluma Dental Associates
- Stephanie Kahle
- R. Michael Alvarez
- Dr. Robert S. Gilbert
- Professional Dental Laboratories
- Aitchison, Bernard
- Marlinski Richard J
- Gerald E. Di